# VAZHI DAPT v1.1 — Tamil Language Adaptation Training

**Pipeline Step 2 of 3:** Train DAPT on cleaned Tamil data using **instruct** model.

```
Step 1: Data Prep (DONE — Vazhi_DAPT_Data_v1_1.ipynb)
  → Produced: CryptoYogi/vazhi-dapt-tamil-v1_1
    52,664 train blocks + 1,075 val blocks = 55M tokens (NFKC-cleaned)

Step 2 (THIS NOTEBOOK): DAPT Training — Kaggle T4 x2 GPU
  → Input:  Packed dataset from HF + Qwen3-0.6B (instruct)
  → Output: CryptoYogi/qwen3-0.6b-tamil-v1_1 (reusable Tamil instruct base)
           CryptoYogi/qwen3-0.6b-tamil-v1_1-lora (adapter backup)

Step 3: SFT (NEXT)
  → Input:  DAPT'd instruct model + ChatML instruction pairs
  → Output: Final VAZHI model
```

**Key changes from v1.0 (incorporating multi-agent review + GPT5.2 feedback):**
1. **Instruct model**, not Base — preserves chat capabilities, DAPT adds Tamil fluency
2. **Higher LR** (5e-5 vs 2e-5) — faster learning for instruct model adaptation
3. **Cleaner data** — NFKC normalized, 70% Tamil threshold, \ufffd stripped
4. **3.4x more tokens** — 55M vs 16M (Sangraha verified only was sufficient)
5. **No step cap** — train ~1 full epoch over entire dataset
6. **Dual T4** — Trainer auto-uses both GPUs via DataParallel (~2x throughput)
7. **Suppress `<think>` tokens** during eval (Qwen3 instruct artifact)
8. **Better eval** — perplexity + word validity + side-by-side with vanilla instruct
9. **Checkpoint resume** — can resume from checkpoint if Kaggle disconnects

**Target:** Kaggle T4 x2 (2×15GB) | ~1,645 steps | Est. ~7 hours with dual GPU

## 1. Install Dependencies

**After running this cell, RESTART the session** (Runtime → Restart session)

In [1]:
!pip install -q -U \
  "transformers>=4.45.0,<5.0.0" \
  "accelerate>=0.34.2" \
  "peft>=0.12.0" \
  "datasets>=2.21.0" \
  "huggingface_hub>=0.24.7"

print("\u2705 Dependencies installed")
print("\u26a0\ufe0f  RESTART THE SESSION NOW (Runtime \u2192 Restart session)")

✅ Dependencies installed
⚠️  RESTART THE SESSION NOW (Runtime → Restart session)


## 2. Configuration

In [2]:
# Multi-GPU setup — Kaggle T4 x2
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import json
import re
import random
import glob
import gc
import torch
import numpy as np
from dataclasses import dataclass
from collections import Counter
from datasets import load_dataset
from huggingface_hub import login, HfApi

from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    TrainerCallback, Trainer, TrainingArguments,
)
from peft import LoraConfig, get_peft_model, PeftModel

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# === KEY CONFIG ===
BASE_MODEL = "Qwen/Qwen3-0.6B"       # INSTRUCT model (v1.0 used Base — that was wrong)
DATASET_NAME = "CryptoYogi/vazhi-dapt-tamil-v1_1"  # Pre-built by Data Prep v1.1
OUTPUT_MODEL = "CryptoYogi/qwen3-0.6b-tamil-v1_1"  # Merged output
ADAPTER_REPO = "CryptoYogi/qwen3-0.6b-tamil-v1_1-lora"  # Adapter backup

# Training config
MAX_SEQ_LENGTH = 1024        # Must match data prep notebook
LEARNING_RATE = 5e-5         # 2.5x higher than v1.0 (2e-5 was too gentle)
LORA_R = 16                  # Conservative rank for 0.6B
LORA_ALPHA = 32
BATCH_SIZE = 4               # Per-device batch size
GRADIENT_ACCUMULATION = 4    # Halved from 8 — 2 GPUs compensate: 4 x 2 x 4 = 32 effective
WARMUP_RATIO = 0.05
MAX_EPOCHS = 1               # Full single epoch over 55M tokens

# Qwen3 instruct <think> tokens to suppress during generation
THINK_TOKEN_IDS = [151667, 151668]  # <think>, </think>

# GPU setup
n_gpus = torch.cuda.device_count()

print(f"\u2705 Configuration loaded")
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA: {torch.cuda.is_available()}")
print(f"   GPUs: {n_gpus}")
for i in range(n_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    gpu_mem = torch.cuda.get_device_properties(i).total_mem / 1024**3 if hasattr(torch.cuda.get_device_properties(i), 'total_mem') else torch.cuda.get_device_properties(i).total_memory / 1024**3
    print(f"   GPU {i}: {gpu_name} ({gpu_mem:.0f} GB)")

effective_batch = BATCH_SIZE * n_gpus * GRADIENT_ACCUMULATION
print()
print(f"\U0001f4cb DAPT Training v1.1:")
print(f"   Base model:  {BASE_MODEL} (INSTRUCT \u2014 preserves chat capability)")
print(f"   Dataset:     {DATASET_NAME}")
print(f"   Output:      {OUTPUT_MODEL}")
print(f"   LR:          {LEARNING_RATE} (2.5x higher than v1.0)")
print(f"   LoRA:        r={LORA_R}, alpha={LORA_ALPHA}")
print(f"   Batch:       {BATCH_SIZE} x {n_gpus} GPUs x {GRADIENT_ACCUMULATION} accum = {effective_batch} effective")
print(f"   Epochs:      {MAX_EPOCHS}")
print(f"   fp16:        True")
print(f"   Grad ckpt:   True")
print(f"   <think> suppress: IDs {THINK_TOKEN_IDS}")

2026-02-13 05:56:37.759481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770962197.780326     120 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770962197.786667     120 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770962197.803046     120 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770962197.803066     120 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770962197.803069     120 computation_placer.cc:177] computation placer alr

✅ Configuration loaded
   PyTorch: 2.8.0+cu126
   CUDA: True
   GPUs: 2
   GPU 0: Tesla T4 (15 GB)
   GPU 1: Tesla T4 (15 GB)

📋 DAPT Training v1.1:
   Base model:  Qwen/Qwen3-0.6B (INSTRUCT — preserves chat capability)
   Dataset:     CryptoYogi/vazhi-dapt-tamil-v1_1
   Output:      CryptoYogi/qwen3-0.6b-tamil-v1_1
   LR:          5e-05 (2.5x higher than v1.0)
   LoRA:        r=16, alpha=32
   Batch:       4 x 2 GPUs x 4 accum = 32 effective
   Epochs:      1
   fp16:        True
   Grad ckpt:   True
   <think> suppress: IDs [151667, 151668]


In [3]:
# Login to HuggingFace
# Kaggle: from kaggle_secrets import UserSecretsClient; login(token=UserSecretsClient().get_secret("HF_TOKEN"))
# Colab: login()
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
hf_token = secrets.get_secret("HF_TOKEN")
login(token=hf_token)
print("\u2705 Logged in to HuggingFace")

✅ Logged in to HuggingFace


## 3. Load Pre-Built Dataset

Created by `Vazhi_DAPT_Data_v1_1.ipynb` (already run on Colab):
- Sangraha **verified Tamil only** (93% keep rate — highest quality)
- NFKC normalized, \ufffd stripped, zero-width chars removed
- Tamil >= 70%, dedup, repetition filtered
- 27,105 docs → 53,739 blocks of 1024 tokens → 55M tokens
- Train: 52,664 blocks | Validation: 1,075 blocks

In [4]:
print(f"\U0001f4e5 Loading pre-built dataset from {DATASET_NAME}...")
ds = load_dataset(DATASET_NAME)

train_dataset = ds["train"]
eval_dataset = ds["validation"]

print(f"\u2705 Dataset loaded:")
print(f"   Train:      {len(train_dataset):,} blocks")
print(f"   Validation: {len(eval_dataset):,} blocks")
print(f"   Block size: {len(train_dataset[0]['input_ids'])} tokens")
print(f"   Columns:    {train_dataset.column_names}")

total_train_tokens = len(train_dataset) * MAX_SEQ_LENGTH
print(f"   Total train tokens: {total_train_tokens:,}")

# Verify block size matches our config
assert len(train_dataset[0]["input_ids"]) == MAX_SEQ_LENGTH, \
    f"Block size mismatch: dataset has {len(train_dataset[0]['input_ids'])}, config has {MAX_SEQ_LENGTH}"
print("\u2705 Block size verified")

📥 Loading pre-built dataset from CryptoYogi/vazhi-dapt-tamil-v1_1...


README.md:   0%|          | 0.00/475 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/53.1M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/53.2M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52664 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1075 [00:00<?, ? examples/s]

✅ Dataset loaded:
   Train:      52,664 blocks
   Validation: 1,075 blocks
   Block size: 1024 tokens
   Columns:    ['input_ids', 'attention_mask', 'labels']
   Total train tokens: 53,927,936
✅ Block size verified


## 4. Load Tokenizer

In [5]:
print(f"\U0001f4e5 Loading tokenizer from {BASE_MODEL}...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.padding_side = "right"

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"\u2705 Tokenizer ready: {len(tokenizer)} tokens")
print(f"   eos_token: {tokenizer.eos_token!r} (ID {tokenizer.eos_token_id})")
print(f"   pad_token: {tokenizer.pad_token!r} (ID {tokenizer.pad_token_id})")

# Verify <think> token IDs
for tid in THINK_TOKEN_IDS:
    token_str = tokenizer.decode([tid])
    print(f"   Token {tid}: {token_str!r}")

# === HELPER FUNCTIONS ===
def count_tamil_chars(text):
    return sum(1 for c in text if '\u0B80' <= c <= '\u0BFF')

def tamil_char_pct(text):
    if not text:
        return 0.0
    return 100.0 * count_tamil_chars(text) / len(text)

def count_tamil_words(text):
    """Count words that are predominantly Tamil characters."""
    words = text.split()
    tamil_words = 0
    for w in words:
        if len(w) > 0 and tamil_char_pct(w) > 50:
            tamil_words += 1
    return tamil_words, len(words)

# Quick sanity: decode a sample from the dataset
sample_text = tokenizer.decode(train_dataset[0]["input_ids"][:100])
print(f"\n\U0001f50d Sample from dataset (first 100 tokens):")
print(f"   Tamil%: {tamil_char_pct(sample_text):.0f}%")
print(f"   Text:   {sample_text[:200]}...")

📥 Loading tokenizer from Qwen/Qwen3-0.6B...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

✅ Tokenizer ready: 151669 tokens
   eos_token: '<|im_end|>' (ID 151645)
   pad_token: '<|endoftext|>' (ID 151643)
   Token 151667: '<think>'
   Token 151668: '</think>'

🔍 Sample from dataset (first 100 tokens):
   Tamil%: 86%
   Text:   ித்தவர். அவர் ஒட்டலில் சுத்தம் செய்யும் பணியில் ஈடுபட்டவர். அவரது குடும்பம் பண வசதி இல்லாத குடும்ப...


## 5. Load Instruct Model + LoRA Setup

**v1.1 change:** Using **Instruct** model instead of Base.

Rationale (from multi-agent review):
- Base model produced gibberish even after DAPT — no Tamil knowledge to build on
- Instruct model already has some language capability
- DAPT on instruct preserves chat behaviors while deepening Tamil fluency
- `<think>` tokens suppressed during generation, not a problem during training

In [6]:
# fp16 — no 4-bit quantization!
# Lesson from v1.0: 4-bit bypasses Tensor Cores on T4, 3x slower
# Qwen3-0.6B in fp16 = ~1.2GB per GPU, fits easily on T4 x2

print(f"\U0001f4e5 Loading {BASE_MODEL} in fp16 (INSTRUCT model)...")
print(f"   NO device_map — Trainer handles DataParallel across {n_gpus} GPUs")

# IMPORTANT: Do NOT use device_map here!
# device_map={"":0} sets model.hf_device_map which makes Trainer think
# the model is already model-parallel → skips DataParallel wrapping → only 1 GPU used.
# v1.0 Trainer output confirmed this: "The model is already on multiple devices. Skipping..."
# Instead: load to cuda:0 manually, let Trainer wrap with DataParallel for both GPUs.
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
model = model.to("cuda:0")

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.use_cache = False  # Disabled for training (incompatible with grad ckpt)

# Enable gradient checkpointing for memory safety
model.gradient_checkpointing_enable()

# Verify no hf_device_map attribute (would prevent DataParallel)
has_device_map = hasattr(model, "hf_device_map")
print(f"   hf_device_map present: {has_device_map} (must be False for DataParallel)")
if has_device_map:
    print(f"   ⚠️  WARNING: hf_device_map detected — Trainer will SKIP DataParallel!")

mem_gb = torch.cuda.memory_allocated(0) / 1024**3
print(f"\u2705 Model loaded in fp16: {model.num_parameters():,} params")
print(f"   GPU 0 memory used: {mem_gb:.1f} GB")
print(f"   This is the INSTRUCT model (has chat capabilities)")
print(f"   Trainer will use DataParallel across {n_gpus} GPUs")

📥 Loading Qwen/Qwen3-0.6B in fp16 (INSTRUCT model)...
   NO device_map — Trainer handles DataParallel across 2 GPUs


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

   hf_device_map present: False (must be False for DataParallel)
✅ Model loaded in fp16: 596,049,920 params
   GPU 0 memory used: 1.1 GB
   This is the INSTRUCT model (has chat capabilities)
   Trainer will use DataParallel across 2 GPUs


In [7]:
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

mem_gb = torch.cuda.memory_allocated() / 1024**3
print(f"\u2705 LoRA applied | GPU: {mem_gb:.1f} GB")

trainable params: 10,092,544 || all params: 606,142,464 || trainable%: 1.6650
✅ LoRA applied | GPU: 1.1 GB


## 6. Compute Training Steps

52,664 train blocks ÷ 32 effective batch (4 per-device × 2 GPUs × 4 accum) = **~1,645 steps** per epoch.
With dual T4 DataParallel, ~4 steps/min → **~7 hours**. Should fit in one Kaggle session.

In [8]:
tokens_per_step = BATCH_SIZE * n_gpus * MAX_SEQ_LENGTH * GRADIENT_ACCUMULATION
steps_per_epoch = len(train_dataset) // (BATCH_SIZE * n_gpus * GRADIENT_ACCUMULATION)
total_steps = steps_per_epoch * MAX_EPOCHS
total_tokens_trained = total_steps * tokens_per_step

# Save/log intervals — more frequent checkpoints for Kaggle disconnect protection
save_steps = max(total_steps // 6, 50)    # ~6 checkpoints per run
log_steps = max(total_steps // 60, 10)    # ~60 log entries
eval_steps = max(total_steps // 10, 25)   # ~10 evals

# Estimate time (~2 steps/min per GPU on T4, with 2 GPUs Trainer uses DataParallel)
est_steps_per_min = 2 * n_gpus  # ~4 steps/min with dual T4
est_hours = total_steps / (est_steps_per_min * 60)

print(f"\U0001f4ca Training Plan:")
print(f"   Dataset tokens:      {len(train_dataset) * MAX_SEQ_LENGTH:,}")
print(f"   Tokens/step:         {tokens_per_step:,}")
print(f"   Steps/epoch:         {steps_per_epoch:,}")
print(f"   Total steps:         {total_steps:,}")
print(f"   Tokens trained:      {total_tokens_trained:,}")
print(f"   Save every:          {save_steps} steps")
print(f"   Log every:           {log_steps} steps")
print(f"   Eval every:          {eval_steps} steps")
print(f"\n   \u23f1\ufe0f Est. time: {est_hours:.1f} hours ({n_gpus} GPUs, ~{est_steps_per_min} steps/min)")

if est_hours > 10:
    print(f"   \u26a0\ufe0f  May be tight for Kaggle session limit (12h). Checkpoints saved every {save_steps} steps.")
    print(f"   If disconnected, use checkpoint resume in Section 7a.")
else:
    print(f"   \u2705 Should fit within a single Kaggle session (12h limit)")

📊 Training Plan:
   Dataset tokens:      53,927,936
   Tokens/step:         32,768
   Steps/epoch:         1,645
   Total steps:         1,645
   Tokens trained:      53,903,360
   Save every:          274 steps
   Log every:           27 steps
   Eval every:          164 steps

   ⏱️ Est. time: 6.9 hours (2 GPUs, ~4 steps/min)
   ✅ Should fit within a single Kaggle session (12h limit)


## 7. Train

If Kaggle disconnects mid-training, skip to **Section 7a** to resume from checkpoint.

In [9]:
# === LOSS LOGGING ===
class LossLoggingCallback(TrainerCallback):
    def __init__(self):
        self.losses = []
        self.eval_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            if "loss" in logs:
                step = state.global_step
                loss = logs["loss"]
                lr = logs.get("learning_rate", 0)
                self.losses.append((step, loss))
                print(f"  Step {step:4d}/{total_steps} | Loss: {loss:.4f} | LR: {lr:.2e}")
            if "eval_loss" in logs:
                eval_loss = logs["eval_loss"]
                ppl = np.exp(min(eval_loss, 20))
                self.eval_losses.append((state.global_step, eval_loss))
                print(f"  \U0001f4ca Eval Loss: {eval_loss:.4f} | Perplexity: {ppl:.1f}")

loss_callback = LossLoggingCallback()

# === DATA COLLATOR ===
@dataclass
class PackedDataCollator:
    """Collator for pre-packed, pre-tokenized sequences."""
    def __call__(self, features):
        return {
            "input_ids": torch.tensor([f["input_ids"] for f in features], dtype=torch.long),
            "attention_mask": torch.tensor([f["attention_mask"] for f in features], dtype=torch.long),
            "labels": torch.tensor([f["labels"] for f in features], dtype=torch.long),
        }

# === TRAINER ===
OUTPUT_DIR = "/kaggle/working/vazhi-dapt-v1_1"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=MAX_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,
    logging_steps=log_steps,
    save_steps=save_steps,
    eval_steps=eval_steps,
    eval_strategy="steps",
    save_total_limit=3,
    fp16=True,
    bf16=False,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    max_grad_norm=1.0,
    optim="adamw_torch",
    report_to="none",
    seed=RANDOM_SEED,
    load_best_model_at_end=False,
    dataloader_pin_memory=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=PackedDataCollator(),
    callbacks=[loss_callback],
)

print("\u2705 Trainer ready")
print(f"   Steps: ~{total_steps} | LR: {LEARNING_RATE} | Effective BS: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"   fp16: True | grad_ckpt: True | optimizer: AdamW (torch)")
print(f"   Model: INSTRUCT (Qwen3-0.6B)")

✅ Trainer ready
   Steps: ~1645 | LR: 5e-05 | Effective BS: 16
   fp16: True | grad_ckpt: True | optimizer: AdamW (torch)
   Model: INSTRUCT (Qwen3-0.6B)


In [10]:
print("\U0001f680 Starting DAPT v1.1 training...")
print(f"   ~{total_steps} steps, fp16=True, INSTRUCT model")
print(f"   Tokens: ~{total_steps * tokens_per_step / 1e6:.0f}M")
print(f"   Est. time: {est_hours:.1f} hours")
print()

train_result = trainer.train()

print("\n\u2705 Training complete!")
metrics = train_result.metrics
for k, v in metrics.items():
    print(f"   {k}: {v}")

# Final eval
print("\n\U0001f4ca Final eval on held-out blocks...")
eval_metrics = trainer.evaluate()
final_eval_loss = eval_metrics.get("eval_loss", float("inf"))
final_eval_ppl = np.exp(min(final_eval_loss, 20))
print(f"   Eval Loss:       {final_eval_loss:.4f}")
print(f"   Eval Perplexity: {final_eval_ppl:.1f}")

# Loss summary
if loss_callback.losses:
    start_loss = loss_callback.losses[0][1]
    end_loss = loss_callback.losses[-1][1]
    print(f"\n\U0001f4c8 Loss: {start_loss:.4f} \u2192 {end_loss:.4f} ({100*(start_loss - end_loss)/start_loss:.1f}% drop)")
if loss_callback.eval_losses:
    first_eval = loss_callback.eval_losses[0][1]
    last_eval = loss_callback.eval_losses[-1][1]
    print(f"   Eval:  {first_eval:.4f} \u2192 {last_eval:.4f} ({100*(first_eval - last_eval)/first_eval:.1f}% drop)")

🚀 Starting DAPT v1.1 training...
   ~1645 steps, fp16=True, INSTRUCT model
   Tokens: ~54M
   Est. time: 6.9 hours



Step,Training Loss,Validation Loss
164,1.100100,1.095042
328,1.043300,1.046491
492,1.028900,1.021160
656,1.006700,1.003887
820,0.987600,0.992039
984,0.996900,0.983142
1148,0.981200,0.976714
1312,0.981700,0.972855
1476,0.962700,0.971036
1640,0.963800,0.970709


  Step   27/1645 | Loss: 1.4268 | LR: 1.57e-05
  Step   54/1645 | Loss: 1.2864 | LR: 3.19e-05
  Step   81/1645 | Loss: 1.1952 | LR: 4.82e-05
  Step  108/1645 | Loss: 1.1533 | LR: 5.00e-05
  Step  135/1645 | Loss: 1.1227 | LR: 4.99e-05
  Step  162/1645 | Loss: 1.1001 | LR: 4.97e-05
  📊 Eval Loss: 1.0950 | Perplexity: 3.0
  Step  189/1645 | Loss: 1.0927 | LR: 4.94e-05
  Step  216/1645 | Loss: 1.0769 | LR: 4.91e-05
  Step  243/1645 | Loss: 1.0655 | LR: 4.87e-05
  Step  270/1645 | Loss: 1.0659 | LR: 4.83e-05
  Step  297/1645 | Loss: 1.0541 | LR: 4.77e-05
  Step  324/1645 | Loss: 1.0433 | LR: 4.71e-05
  📊 Eval Loss: 1.0465 | Perplexity: 2.8
  Step  351/1645 | Loss: 1.0435 | LR: 4.65e-05
  Step  378/1645 | Loss: 1.0385 | LR: 4.58e-05
  Step  405/1645 | Loss: 1.0334 | LR: 4.50e-05
  Step  432/1645 | Loss: 1.0350 | LR: 4.41e-05
  Step  459/1645 | Loss: 1.0185 | LR: 4.32e-05
  Step  486/1645 | Loss: 1.0289 | LR: 4.23e-05
  📊 Eval Loss: 1.0212 | Perplexity: 2.8
  Step  513/1645 | Loss: 1.0296 | 

  📊 Eval Loss: 0.9707 | Perplexity: 2.6
   Eval Loss:       0.9707
   Eval Perplexity: 2.6

📈 Loss: 1.4268 → 0.9638 (32.5% drop)
   Eval:  1.0950 → 0.9707 (11.4% drop)


## 7a. Resume From Checkpoint (if Kaggle disconnected)

**Only run this cell if training was interrupted.** Skip if training completed above.

Find the latest checkpoint and resume training.

In [11]:
# === UNCOMMENT AND RUN ONLY IF TRAINING WAS INTERRUPTED ===
# This cell resumes from the latest checkpoint in OUTPUT_DIR

# import glob
# checkpoints = sorted(glob.glob(f"{OUTPUT_DIR}/checkpoint-*"), key=os.path.getmtime)
# if checkpoints:
#     latest_ckpt = checkpoints[-1]
#     print(f"\U0001f504 Resuming from {latest_ckpt}")
#     train_result = trainer.train(resume_from_checkpoint=latest_ckpt)
#     print("\u2705 Training resumed and completed!")
#     metrics = train_result.metrics
#     for k, v in metrics.items():
#         print(f"   {k}: {v}")
#
#     # Final eval after resume
#     eval_metrics = trainer.evaluate()
#     final_eval_loss = eval_metrics.get("eval_loss", float("inf"))
#     final_eval_ppl = np.exp(min(final_eval_loss, 20))
#     print(f"   Eval Loss:       {final_eval_loss:.4f}")
#     print(f"   Eval Perplexity: {final_eval_ppl:.1f}")
# else:
#     print("\u274c No checkpoints found. Run training from scratch (Section 7).")

## 8. Save & Upload LoRA Adapter

In [12]:
ADAPTER_PATH = "/kaggle/working/vazhi-dapt-v1_1-lora"

print("\U0001f4be Saving LoRA adapter...")
trainer.save_model(ADAPTER_PATH)
tokenizer.save_pretrained(ADAPTER_PATH)

adapter_files = glob.glob(f"{ADAPTER_PATH}/*")
print(f"   Files: {[os.path.basename(f) for f in adapter_files]}")
assert any('adapter' in f for f in adapter_files), "No adapter files!"
print("\u2705 Adapter saved")

# Upload adapter backup
api = HfApi()
api.create_repo(ADAPTER_REPO, exist_ok=True)
print(f"\U0001f4e4 Uploading adapter to {ADAPTER_REPO}...")
api.upload_folder(
    folder_path=ADAPTER_PATH,
    repo_id=ADAPTER_REPO,
    commit_message=f"DAPT v1.1 adapter: Sangraha Tamil (NFKC), instruct model, r={LORA_R}, lr={LEARNING_RATE}",
)
print(f"\u2705 Adapter uploaded: https://huggingface.co/{ADAPTER_REPO}")

💾 Saving LoRA adapter...
   Files: ['tokenizer_config.json', 'README.md', 'special_tokens_map.json', 'chat_template.jinja', 'added_tokens.json', 'vocab.json', 'tokenizer.json', 'adapter_model.safetensors', 'adapter_config.json', 'merges.txt', 'training_args.bin']
✅ Adapter saved
📤 Uploading adapter to CryptoYogi/qwen3-0.6b-tamil-v1_1-lora...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✅ Adapter uploaded: https://huggingface.co/CryptoYogi/qwen3-0.6b-tamil-v1_1-lora


## 9. Merge LoRA in FP16

**Hard rule (Lesson #39):** NEVER merge into 4-bit. Reload base in fp16, merge there.

In [13]:
# Free training model to make room for merge
del model
del trainer
gc.collect()
torch.cuda.empty_cache()
print("\U0001f5d1\ufe0f Training model freed")

# Reload fresh base in fp16 for clean merge
print(f"\U0001f517 Loading fresh {BASE_MODEL} in fp16 for merge...")
base_model_fp16 = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map={"":0},
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(base_model_fp16, ADAPTER_PATH)
peft_model.gradient_checkpointing_disable()
peft_model.config.use_cache = True
peft_model.eval()

print("\U0001f500 Merging LoRA in fp16...")
merged_model = peft_model.merge_and_unload()
print(f"\u2705 Merged: {merged_model.num_parameters():,} params")

🗑️ Training model freed
🔗 Loading fresh Qwen/Qwen3-0.6B in fp16 for merge...
🔀 Merging LoRA in fp16...
✅ Merged: 596,049,920 params


## 10. DAPT Evaluation — Quality Metrics

**v1.1 improvements over v1.0:**
1. **Perplexity** on held-out blocks (measures language modeling quality)
2. **Tamil word validity** — not just char %, but actual word-level quality
3. **Side-by-side with vanilla instruct** — did DAPT actually help?
4. **`<think>` token suppression** via `bad_words_ids`

This is an **instruct** model — it may try to generate in chat format, but for DAPT eval
we test raw text continuation (same as v1.0 for comparability).

In [14]:
merged_model.eval()
merged_model.config.use_cache = True

# Compute perplexity on held-out validation blocks
print("\U0001f4ca Computing perplexity on validation set...")
eval_losses = []
n_eval_samples = min(200, len(eval_dataset))  # Cap for speed

for i in range(n_eval_samples):
    input_ids = torch.tensor([eval_dataset[i]["input_ids"]], dtype=torch.long).to(merged_model.device)
    labels = input_ids.clone()
    with torch.no_grad():
        outputs = merged_model(input_ids=input_ids, labels=labels)
        eval_losses.append(outputs.loss.item())

avg_eval_loss = np.mean(eval_losses)
eval_ppl = np.exp(min(avg_eval_loss, 20))

print(f"   Samples:    {n_eval_samples}")
print(f"   Avg Loss:   {avg_eval_loss:.4f}")
print(f"   Perplexity: {eval_ppl:.1f}")

if eval_ppl < 5:
    print(f"   \u2705 Excellent — low perplexity indicates strong Tamil fluency")
elif eval_ppl < 15:
    print(f"   \u2705 Good — reasonable perplexity for adapted model")
elif eval_ppl < 50:
    print(f"   \u26a0\ufe0f  Moderate — DAPT helped somewhat but room for improvement")
else:
    print(f"   \u274c High perplexity — DAPT may not have converged")

📊 Computing perplexity on validation set...
   Samples:    200
   Avg Loss:   0.9993
   Perplexity: 2.7
   ✅ Excellent — low perplexity indicates strong Tamil fluency


In [15]:
# === GENERATION EVAL ===
eval_prompts = [
    ("prose", "\u0ba4\u0bae\u0bbf\u0bb4\u0bcd\u0ba8\u0bbe\u0b9f\u0bc1 \u0b87\u0ba8\u0bcd\u0ba4\u0bbf\u0baf\u0bbe\u0bb5\u0bbf\u0ba9\u0bcd \u0ba4\u0bc6\u0ba9\u0bcd \u0baa\u0b95\u0bc1\u0ba4\u0bbf\u0baf\u0bbf\u0bb2\u0bcd \u0b85\u0bae\u0bc8\u0ba8\u0bcd\u0ba4\u0bc1\u0bb3\u0bcd\u0bb3 \u0b92\u0bb0\u0bc1 \u0bae\u0bbe\u0ba8\u0bbf\u0bb2\u0bae\u0bcd."),
    ("prose", "\u0baa\u0bca\u0b99\u0bcd\u0b95\u0bb2\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bb0\u0bcd\u0b95\u0bb3\u0bbf\u0ba9\u0bcd \u0bae\u0bc1\u0b95\u0bcd\u0b95\u0bbf\u0baf \u0ba4\u0bbf\u0bb0\u0bc1\u0ba8\u0bbe\u0bb3\u0bcd."),
    ("literature", "\u0bb5\u0bb3\u0bcd\u0bb3\u0bc1\u0bb5\u0bb0\u0bcd \u0b95\u0bc2\u0bb1\u0bbf\u0baf \u0b85\u0bb1\u0bae\u0bcd, \u0baa\u0bca\u0bb0\u0bc1\u0bb3\u0bcd, \u0b87\u0ba9\u0bcd\u0baa\u0bae\u0bcd \u0b8e\u0ba9\u0bcd\u0bb1 \u0bae\u0bc2\u0ba9\u0bcd\u0bb1\u0bc1"),
    ("knowledge", "\u0b9a\u0bbf\u0ba4\u0bcd\u0ba4 \u0bae\u0bb0\u0bc1\u0ba4\u0bcd\u0ba4\u0bc1\u0bb5\u0bae\u0bcd \u0b8e\u0ba9\u0bcd\u0baa\u0ba4\u0bc1 \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0bae\u0b95\u0bcd\u0b95\u0bb3\u0bbf\u0ba9\u0bcd \u0baa\u0bbe\u0bb0\u0bae\u0bcd\u0baa\u0bb0\u0bbf\u0baf"),
    ("daily", "\u0b95\u0bbe\u0bb2\u0bc8\u0baf\u0bbf\u0bb2\u0bcd \u0b8e\u0bb4\u0bc1\u0ba8\u0bcd\u0ba4\u0ba4\u0bc1\u0bae\u0bcd \u0bae\u0bc1\u0ba4\u0bb2\u0bbf\u0bb2\u0bcd"),
    ("short", "\u0ba4\u0bae\u0bbf\u0bb4\u0bcd"),
    ("short", "\u0ba8\u0ba9\u0bcd\u0bb1\u0bbf"),
    ("mixed", "India has many languages. \u0ba4\u0bae\u0bbf\u0bb4\u0bcd is one of the"),
]

print(f"\n{'='*60}")
print(f"\U0001f9ea DAPT v1.1 EVAL: {len(eval_prompts)} Tamil text continuations")
print(f"   (Instruct model \u2014 <think> suppressed with bad_words_ids)")
print(f"{'='*60}")

eval_results = []

for category, prompt_text in eval_prompts:
    inputs = tokenizer(prompt_text, return_tensors="pt").to(merged_model.device)

    with torch.no_grad():
        outputs = merged_model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            no_repeat_ngram_size=4,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            bad_words_ids=[[tid] for tid in THINK_TOKEN_IDS],  # Suppress <think> tokens
        )

    generated_ids = outputs[0][inputs["input_ids"].shape[1]:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # Char-level metrics
    t_pct = tamil_char_pct(response)
    words = response.split()
    unique_ratio = len(set(words)) / max(len(words), 1)

    # Word-level Tamil quality (NEW in v1.1)
    tamil_words, total_words = count_tamil_words(response)
    tamil_word_pct = 100.0 * tamil_words / max(total_words, 1)

    # Quality flags
    is_repetitive = unique_ratio < 0.3 and len(words) > 10
    is_empty = len(response.strip()) < 10
    is_code = any(kw in response[:100] for kw in ['def ', 'class ', 'import ', '{"', 'var '])
    has_think = '<think>' in response or '</think>' in response

    status = "\u2705"
    if is_empty: status = "\u274c EMPTY"
    elif is_code: status = "\u274c CODE"
    elif has_think: status = "\u274c THINK"
    elif is_repetitive: status = "\u26a0\ufe0f LOOP"
    elif t_pct < 20 and category != "mixed": status = "\u26a0\ufe0f LOW TAMIL"

    eval_results.append({
        "category": category,
        "prompt": prompt_text,
        "response": response[:300],
        "status": status,
        "tamil_char_pct": t_pct,
        "tamil_word_pct": tamil_word_pct,
        "unique_ratio": unique_ratio,
    })

    print(f"\n[{category.upper()}] {status} (Char: {t_pct:.0f}%, Words: {tamil_word_pct:.0f}%, Unique: {unique_ratio:.0%})")
    print(f"  Prompt: {prompt_text[:60]}")
    print(f"  Output: {response[:300]}")
    print("-" * 50)


🧪 DAPT v1.1 EVAL: 8 Tamil text continuations
   (Instruct model — <think> suppressed with bad_words_ids)

[PROSE] ✅ (Char: 81%, Words: 94%, Unique: 100%)
  Prompt: தமிழ்நாடு இந்தியாவின் தென் பகுதியில் அமைந்துள்ள ஒரு மாநிலம்.
  Output:  எலயூகி, 30-இறப்பு கண்டிருப்பதங்களை உள்ளாட்சி வழங்றுள்஽.
சீனாவோ, ஜெபிஐவோ - தொல்லை!
எலயோடு தேர்வுஃ
அஸ்ஸார் ஓட்டிய போர் திருஷ்டமாக உயர்த�
--------------------------------------------------

[PROSE] ❌ EMPTY (Char: 0%, Words: 0%, Unique: 0%)
  Prompt: பொங்கல் தமிழர்களின் முக்கிய திருநாள்.
  Output: 
--------------------------------------------------

[LITERATURE] ✅ (Char: 76%, Words: 100%, Unique: 100%)
  Prompt: வள்ளுவர் கூறிய அறம், பொருள், இன்பம் என்ற மூன்று
  Output:  சாதகப் பண்ணையின் 3-ஆவது தடை.
--------------------------------------------------

[KNOWLEDGE] ✅ (Char: 80%, Words: 80%, Unique: 100%)
  Prompt: சித்த மருத்துவம் என்பது தமிழ் மக்களின் பாரம்பரிய
  Output: ப்படி 2014-ல், "அறிவோ அழுத்டு" (ஆண.ஜ.) - இயக்கத்தில் நடித்சான் ரங்கநாத், ஆராய்ச்சி உறுப்பி

In [16]:
# === EVAL SUMMARY ===
print(f"\n{'='*60}")
print(f"\U0001f4ca DAPT v1.1 EVAL SUMMARY")
print(f"{'='*60}")

pass_count = sum(1 for r in eval_results if r['status'] == "\u2705")
avg_tamil_char = np.mean([r['tamil_char_pct'] for r in eval_results])
avg_tamil_word = np.mean([r['tamil_word_pct'] for r in eval_results])
avg_unique = np.mean([r['unique_ratio'] for r in eval_results])

print(f"   Passed:         {pass_count}/{len(eval_results)}")
print(f"   Avg Tamil char: {avg_tamil_char:.0f}%")
print(f"   Avg Tamil word: {avg_tamil_word:.0f}% (NEW \u2014 more meaningful than char%)")
print(f"   Avg Unique:     {avg_unique:.0%}")
print(f"   Eval PPL:       {eval_ppl:.1f}")
print()

print(f"{'Status':<10} {'Category':<12} {'Char%':>6} {'Word%':>6} {'Unique':>7}")
print("-" * 50)
for r in eval_results:
    print(f"{r['status']:<10} [{r['category']}] {r['tamil_char_pct']:>5.0f}% {r['tamil_word_pct']:>5.0f}% {r['unique_ratio']:>6.0%}")

print(f"\n\U0001f4cb v1.0 Baseline for comparison:")
print(f"   v1.0: 8/8 pass, avg Tamil char 66%, PPL ~2.8 (base model)")
print(f"   v1.0 comparison: DAPT vs Base was -2% (DAPT didn't help)")

if pass_count >= len(eval_results) * 0.7 and avg_tamil_word > 40:
    print(f"\n\U0001f389 DAPT v1.1 successful! Proceed to SFT.")
elif pass_count >= len(eval_results) * 0.5:
    print(f"\n\u26a0\ufe0f  Partial. May still be better than v1.0 — run comparison notebook.")
else:
    print(f"\n\u274c DAPT v1.1 failed. Check loss curve, data quality.")


📊 DAPT v1.1 EVAL SUMMARY
   Passed:         7/8
   Avg Tamil char: 59%
   Avg Tamil word: 66% (NEW — more meaningful than char%)
   Avg Unique:     88%
   Eval PPL:       2.7

Status     Category      Char%  Word%  Unique
--------------------------------------------------
✅          [prose]    81%    94%   100%
❌ EMPTY    [prose]     0%     0%     0%
✅          [literature]    76%   100%   100%
✅          [knowledge]    80%    80%   100%
✅          [daily]    51%    60%   100%
✅          [short]    73%    74%   100%
✅          [short]    62%    64%   100%
✅          [mixed]    51%    55%   100%

📋 v1.0 Baseline for comparison:
   v1.0: 8/8 pass, avg Tamil char 66%, PPL ~2.8 (base model)
   v1.0 comparison: DAPT vs Base was -2% (DAPT didn't help)

🎉 DAPT v1.1 successful! Proceed to SFT.


## 10a. Side-by-Side: DAPT v1.1 vs Vanilla Instruct

Load vanilla Qwen3-0.6B instruct and run the same prompts.
This tells us if DAPT actually improved Tamil over the base instruct model.

In [17]:
# Free merged model to load vanilla
del merged_model
gc.collect()
torch.cuda.empty_cache()

print(f"\U0001f4e5 Loading vanilla {BASE_MODEL} for comparison...")
vanilla_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map={"":0},
    trust_remote_code=True,
)
vanilla_model.eval()
vanilla_model.config.use_cache = True
print("\u2705 Vanilla instruct model loaded")

vanilla_results = []
for category, prompt_text in eval_prompts:
    inputs = tokenizer(prompt_text, return_tensors="pt").to(vanilla_model.device)
    with torch.no_grad():
        outputs = vanilla_model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            no_repeat_ngram_size=4,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            bad_words_ids=[[tid] for tid in THINK_TOKEN_IDS],
        )
    generated_ids = outputs[0][inputs["input_ids"].shape[1]:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True)
    t_pct = tamil_char_pct(response)
    tamil_words, total_words = count_tamil_words(response)
    tamil_word_pct = 100.0 * tamil_words / max(total_words, 1)
    words = response.split()
    unique_ratio = len(set(words)) / max(len(words), 1)
    vanilla_results.append({
        "category": category,
        "response": response[:300],
        "tamil_char_pct": t_pct,
        "tamil_word_pct": tamil_word_pct,
        "unique_ratio": unique_ratio,
    })

del vanilla_model
gc.collect()
torch.cuda.empty_cache()
print("\U0001f5d1\ufe0f Vanilla model freed")

📥 Loading vanilla Qwen/Qwen3-0.6B for comparison...
✅ Vanilla instruct model loaded
🗑️ Vanilla model freed


In [18]:
# === COMPARISON TABLE ===
print("=" * 80)
print("\U0001f4ca SIDE-BY-SIDE: Vanilla Instruct vs DAPT v1.1")
print("=" * 80)

print(f"\n{'Category':<12} {'Van Char%':>10} {'DAPT Char%':>11} {'Van Word%':>10} {'DAPT Word%':>11} {'Winner':>8}")
print("-" * 80)

dapt_wins_char = 0
dapt_wins_word = 0

for v, d in zip(vanilla_results, eval_results):
    # Use word% as primary metric (more meaningful than char%)
    winner = "DAPT" if d["tamil_word_pct"] > v["tamil_word_pct"] + 5 else (
        "VANILLA" if v["tamil_word_pct"] > d["tamil_word_pct"] + 5 else "TIE"
    )
    if d["tamil_char_pct"] > v["tamil_char_pct"]: dapt_wins_char += 1
    if d["tamil_word_pct"] > v["tamil_word_pct"]: dapt_wins_word += 1

    print(f"{v['category']:<12} {v['tamil_char_pct']:>8.0f}% {d['tamil_char_pct']:>9.0f}% {v['tamil_word_pct']:>8.0f}% {d['tamil_word_pct']:>9.0f}%  {winner:>8}")

print("-" * 80)

avg_van_char = np.mean([r["tamil_char_pct"] for r in vanilla_results])
avg_dapt_char = np.mean([r["tamil_char_pct"] for r in eval_results])
avg_van_word = np.mean([r["tamil_word_pct"] for r in vanilla_results])
avg_dapt_word = np.mean([r["tamil_word_pct"] for r in eval_results])

print(f"{'AVERAGE':<12} {avg_van_char:>8.0f}% {avg_dapt_char:>9.0f}% {avg_van_word:>8.0f}% {avg_dapt_word:>9.0f}%")

print(f"\n\U0001f3c6 DAPT wins (char%): {dapt_wins_char}/{len(eval_prompts)}")
print(f"   DAPT wins (word%): {dapt_wins_word}/{len(eval_prompts)}")
print(f"   Char% change: {avg_van_char:.0f}% \u2192 {avg_dapt_char:.0f}% ({avg_dapt_char - avg_van_char:+.0f}%)")
print(f"   Word% change: {avg_van_word:.0f}% \u2192 {avg_dapt_word:.0f}% ({avg_dapt_word - avg_van_word:+.0f}%)")

if avg_dapt_word > avg_van_word + 10:
    print(f"\n\U0001f389 DAPT v1.1 CLEARLY improved Tamil! Proceed to SFT.")
elif avg_dapt_word > avg_van_word + 3:
    print(f"\n\u2705 DAPT v1.1 shows meaningful improvement. Proceed to SFT.")
elif avg_dapt_word > avg_van_word:
    print(f"\n\u26a0\ufe0f  Marginal improvement. May need more training tokens.")
else:
    print(f"\n\u274c DAPT v1.1 didn't improve Tamil word quality. Investigate.")

📊 SIDE-BY-SIDE: Vanilla Instruct vs DAPT v1.1

Category      Van Char%  DAPT Char%  Van Word%  DAPT Word%   Winner
--------------------------------------------------------------------------------
prose               0%        81%        0%        94%      DAPT
prose               8%         0%        6%         0%   VANILLA
literature          5%        76%        4%       100%      DAPT
knowledge          11%        80%        7%        80%      DAPT
daily               5%        51%        1%        60%      DAPT
short               3%        73%        2%        74%      DAPT
short               0%        62%        0%        64%      DAPT
mixed               0%        51%        0%        55%      DAPT
--------------------------------------------------------------------------------
AVERAGE             4%        59%        3%        66%

🏆 DAPT wins (char%): 7/8
   DAPT wins (word%): 7/8
   Char% change: 4% → 59% (+55%)
   Word% change: 3% → 66% (+63%)

🎉 DAPT v1.1 CLEARLY improved 

In [19]:
# === DETAILED SIDE-BY-SIDE OUTPUT ===
print("=" * 70)
print("\U0001f50d DETAILED OUTPUT COMPARISON")
print("=" * 70)

for v, d in zip(vanilla_results, eval_results):
    print(f"\n\u250c\u2500 [{d['category'].upper()}] Prompt: {d['prompt'][:60]}")
    print(f"\u2502")
    print(f"\u2502 VANILLA (Char {v['tamil_char_pct']:.0f}%, Word {v['tamil_word_pct']:.0f}%):")
    print(f"\u2502   {v['response'][:200]}")
    print(f"\u2502")
    print(f"\u2502 DAPT v1.1 (Char {d['tamil_char_pct']:.0f}%, Word {d['tamil_word_pct']:.0f}%):")
    print(f"\u2502   {d['response'][:200]}")
    print(f"\u2514{'\u2500' * 69}")

🔍 DETAILED OUTPUT COMPARISON

┌─ [PROSE] Prompt: தமிழ்நாடு இந்தியாவின் தென் பகுதியில் அமைந்துள்ள ஒரு மாநிலம்.
│
│ VANILLA (Char 0%, Word 0%):
│    1780-2356, 495 - 2355 (1850)ஆக a time period that is between the end of 17th and beginning of 18th century in India's history.

This text says something about how the Indian state was structured duri
│
│ DAPT v1.1 (Char 81%, Word 94%):
│    எலயூகி, 30-இறப்பு கண்டிருப்பதங்களை உள்ளாட்சி வழங்றுள்஽.
சீனாவோ, ஜெபிஐவோ - தொல்லை!
எலயோடு தேர்வுஃ
அஸ்ஸார் ஓட்டிய போர் திருஷ்டமாக உயர்த�
└─────────────────────────────────────────────────────────────────────

┌─ [PROSE] Prompt: பொங்கல் தமிழர்களின் முக்கிய திருநாள்.
│
│ VANILLA (Char 8%, Word 6%):
│    2019-2023இன் ஆணி - ఇంటு
அதురక്ഷలിയോ, దస್తలസനిயీన അത്, పౌരశృణయ గావడం... ఉద్బ్రచు, ఈ జగధులం లో ఎంపఢులు, 2564-2573 వర్షము
│
│ DAPT v1.1 (Char 0%, Word 0%):
│   
└─────────────────────────────────────────────────────────────────────

┌─ [LITERATURE] Prompt: வள்ளுவர் கூறிய அறம், பொருள், இன்பம் என்ற மூன்று
│
│ VANIL

## 11. Upload Merged Model

In [20]:
# Reload the merged model for upload (was freed for vanilla comparison)
print(f"\U0001f517 Reloading merged model for upload...")
base_model_fp16 = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map={"":0},
    trust_remote_code=True,
)
peft_model = PeftModel.from_pretrained(base_model_fp16, ADAPTER_PATH)
peft_model.eval()
merged_model = peft_model.merge_and_unload()
print(f"\u2705 Merged model ready: {merged_model.num_parameters():,} params")

🔗 Reloading merged model for upload...
✅ Merged model ready: 596,049,920 params


In [21]:
api = HfApi()
api.create_repo(OUTPUT_MODEL, exist_ok=True)

print(f"\U0001f4e4 Pushing merged fp16 model to {OUTPUT_MODEL}...")
merged_model.push_to_hub(
    OUTPUT_MODEL,
    private=False,
    commit_message=(
        f"DAPT v1.1: Tamil-adapted Qwen3-0.6B (instruct), "
        f"Sangraha NFKC-cleaned, LoRA r={LORA_R}, lr={LEARNING_RATE}"
    ),
)
tokenizer.push_to_hub(OUTPUT_MODEL)

print(f"\n\u2705 Model: https://huggingface.co/{OUTPUT_MODEL}")
print(f"\u2705 Adapter: https://huggingface.co/{ADAPTER_REPO}")
print(f"\n\U0001f449 Next: Run SFT notebook with BASE_MODEL = \"{OUTPUT_MODEL}\"")

📤 Pushing merged fp16 model to CryptoYogi/qwen3-0.6b-tamil-v1_1...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            


✅ Model: https://huggingface.co/CryptoYogi/qwen3-0.6b-tamil-v1_1
✅ Adapter: https://huggingface.co/CryptoYogi/qwen3-0.6b-tamil-v1_1-lora

👉 Next: Run SFT notebook with BASE_MODEL = "CryptoYogi/qwen3-0.6b-tamil-v1_1"


## Summary

| Artifact | Repo | Purpose |
|----------|------|---------|
| Packed DAPT data | `CryptoYogi/vazhi-dapt-tamil-v1_1` | Cleaned training data (NFKC, 70% Tamil) |
| Merged fp16 model | `CryptoYogi/qwen3-0.6b-tamil-v1_1` | Reusable Tamil instruct base for SFT |
| LoRA adapter | `CryptoYogi/qwen3-0.6b-tamil-v1_1-lora` | Recovery backup |

### Changes from v1.0
| Dimension | v1.0 | v1.1 |
|-----------|------|------|
| Model | Qwen3-0.6B-**Base** | Qwen3-0.6B (**Instruct**) |
| Data | 16M tokens, Tamil >= 50% | **55M tokens**, Tamil >= 70%, NFKC |
| Sources | Sangraha verified only | Sangraha verified (sufficient at 55M) |
| Docs/Blocks | ~16K / 31,599 | 27,105 / **52,664** |
| LR | 2e-5 | **5e-5** |
| GPUs | T4 x1 (single) | **T4 x2** (DataParallel) |
| Steps | 500 (capped) | **~1,645** (full epoch) |
| Est. time | ~4h | **~7h** (dual T4) |
| Eval | Tamil char% only | Char% + **word%** + perplexity + comparison |

### Next: SFT (Stage 3)
```python
BASE_MODEL = "CryptoYogi/qwen3-0.6b-tamil-v1_1"  # THIS model
DATASET = "CryptoYogi/vazhi-tamil-sft-v4_0"       # ChatML instruction pairs
```

### If DAPT v1.1 failed
1. Check loss curve — did it plateau early? May need higher LR or more epochs
2. Check word% — if char% is high but word% is low, model is generating Tamil-like gibberish
3. Compare with vanilla — if no improvement, data quality may still be an issue
4. Fallback: Sarvam-1 IQ3_M (1.17GB, proven Tamil capability)